# <font color="00ff00">  **Statistical Analysis for Accented Speech recognition**</font>
<font color=white> **evaluates the performance of an ASR model regarding accented speech**</font> 


In [5]:
!pip install hyphenate
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os
import numpy as np
import pickle
import pandas as pd
import json
import re
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Audio
from IPython.display import display
from scipy.io import wavfile
from itertools import chain 
import string
import Levenshtein as Lev
from itertools import groupby
import scipy.stats as st
from scipy import signal
import nltk
from scipy.stats import wasserstein_distance as wd
from sklearn.metrics.pairwise import euclidean_distances as ed
from pyemd import emd
from hyphenate import hyphenate_word
from itertools import islice 
from collections import OrderedDict

In [7]:
# Import the dataset file by method1 
# from google.colab import files
# uploaded = files.upload()

! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/results.json   /content/results.json

! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/results-AOE.json   /content/results-AOE.json

! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/cv-corpus04072022/de/validated.tsv /content/validated.tsv

# copy the expermintations files to deal with them
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_at.txt /content/test_at.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_ca.txt /content/test_ca.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_ch.txt /content/test_ch.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_de_al.txt /content/test_de_al.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_de_ni.txt /content/test_de_ni.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_de.txt /content/test_de.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_fr.txt /content/test_fr.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_gb.txt /content/test_gb.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_it.txt /content/test_it.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_ru.txt /content/test_ru.txt
! cp /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_us.txt /content/test_us.txt

# copy the expermintations files from Mozilla Commen Voice v 10 to deal with them

In [12]:
#@title ###**According to a given utterance, this code shows and writes in which Audio files and which Accent this given utterance is located:**
 
###########################################################################################################################################################
########### creat a Dataframe called "dataset_audiofilename_transcript_accent" contains all audiofilename and thier 'transcript' and 'test_file' /Accent####
###########################################################################################################################################################
list_dataset_test_=['test_at.txt','test_gb.txt','test_it.txt','test_de_al.txt','test_fr.txt','test_de_ni.txt','test_ch.txt',
               'test_de.txt','test_us.txt','test_ca.txt','test_ru.txt']
list_dataset_test_accent=['Österreichisches Deutsch','Britisches Deutsch','Italienisch Deutsch','Alemannische Färbung,Schweizer Standart Deutsch',
'Französisch Deutsch','Niederländisch Deutsch','Schweizerdeutsch','Deutschland Deutsch','Amerikanisches Deutsch','Kanadisches Deutsch','Russisch Deutsch']

data_dict_accent_duration  = dict(zip( list_dataset_test_, list_dataset_test_accent))  


### here is the required_ utterance
required_utterance="findest du dich allein in braunschweig zurecht"
newfile=0
# dict_audiofilename_transcript_accent=[]
# To find out the Audio file's Accent
list_test_accent_txt_values=[]
list_test_accent_txt_keys=[]
list_test_file=[]
list_accent_long=[]

if __name__ == "__main__":
  Dict_results = json.load(open('results.json'))
  for test_file in Dict_results:
    # print(test_file)
    list_test_accent_txt_values_temp = [v["reference"] for v in Dict_results[test_file].values()]
    list_test_accent_txt_keys_temp=[re.split(r'[.|/]',k)[9] for k in Dict_results[test_file].keys()]
    list_test_file_temp=[test_file for v in Dict_results[test_file].values()]

# creat a Dataframe called  dataset_audiofilename_transcript_accent contains all audiofilename and thier 'transcript' and 'test_file' /Accent
    list_test_accent_txt_values.extend(list_test_accent_txt_values_temp)
    list_test_accent_txt_keys.extend(list_test_accent_txt_keys_temp)
    list_test_file.extend(list_test_file_temp)
    keys=['audiofilename','transcript','test_file']
    trans_dict_test_file_result = dict(zip(keys,[list_test_accent_txt_keys, list_test_accent_txt_values,list_test_file]))
    
dataset_audiofilename_transcript_accent = pd.DataFrame(trans_dict_test_file_result)

#############################################################################################
##### Show and write which Audio files and which Accent the given word is located ###########

for key, value in dataset_audiofilename_transcript_accent['transcript'].items():
  if required_utterance in value:

# to show the full name accent of the founded result 
    for keys_accent_long_i, values_accent_long_i in data_dict_accent_duration.items():
      if dataset_audiofilename_transcript_accent.test_file[key]==keys_accent_long_i:
        print()
        # print(dataset_audiofilename_transcript_accent.audiofilename[key],dataset_audiofilename_transcript_accent.transcript[key],
        # f'*** Accent ={dataset_audiofilename_transcript_accent.test_file[key]}')
        print(dataset_audiofilename_transcript_accent.audiofilename[key],dataset_audiofilename_transcript_accent.transcript[key],
              f'*** Accent is {data_dict_accent_duration[keys_accent_long_i]}')
 

        # # Alternative way (for speed performance) to export the Audio files numbers and thiers Acccent to an external file
        # # os.mkdir('/content/audiofilenames_transcript')

        # if newfile==0:
        #   dataset_accent_write_file = open(f'audiofilenames_transcript_accent_of_an_utterance.txt','w')
        #   newfile=1
        # else:
        #   dataset_accent_write_file = open(f'audiofilenames_transcript_accent_of_an_utterance.txt','a')
        #   dataset_accent_write_file.write(dataset_audiofilename_transcript_accent.audiofilename[key],dataset_audiofilename_transcript_accent.transcript[key],
        #       f'*** Accent is {data_dict_accent_duration[keys_accent_long_i]}\n')
        #   dataset_accent_write_file.close()
        #   # ! cp /content/audiofilenames_transcript/*.txt /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_data/audiofilenames_transcript/


common_voice_de_17686048 findest du dich allein in braunschweig zurecht *** Accent is Schweizerdeutsch

common_voice_de_17650552 findest du dich allein in braunschweig zurecht *** Accent is Deutschland Deutsch


In [13]:
#@title ###**According to a given word, this code shows and writes in which Audio files and which Accent this given word is located:**

###########################################################################################################################################################
########### creat a Dataframe called "dataset_audiofilename_transcript_accent" contains all audiofilename and thier 'transcript' and 'test_file' /Accent####
###########################################################################################################################################################
list_dataset_test_=['test_at.txt','test_gb.txt','test_it.txt','test_de_al.txt','test_fr.txt','test_de_ni.txt','test_ch.txt',
               'test_de.txt','test_us.txt','test_ca.txt','test_ru.txt']
list_dataset_test_accent=['Österreichisches Deutsch','Britisches Deutsch','Italienisch Deutsch','Alemannische Färbung,Schweizer Standart Deutsch',
'Französisch Deutsch','Niederländisch Deutsch','Schweizerdeutsch','Deutschland Deutsch','Amerikanisches Deutsch','Kanadisches Deutsch','Russisch Deutsch']

data_dict_accent_duration  = dict(zip( list_dataset_test_, list_dataset_test_accent))  


### here is the required_word
required_word="später"
newfile=0
# dict_audiofilename_transcript_accent=[]
# To find out the Audio file's Accent
list_test_accent_txt_values=[]
list_test_accent_txt_keys=[]
list_test_file=[]
list_accent_long=[]

if __name__ == "__main__":
  Dict_results = json.load(open('results.json'))
  for test_file in Dict_results:
    # print(test_file)
    list_test_accent_txt_values_temp = [v["reference"] for v in Dict_results[test_file].values()]
    list_test_accent_txt_keys_temp=[re.split(r'[.|/]',k)[9] for k in Dict_results[test_file].keys()]
    list_test_file_temp=[test_file for v in Dict_results[test_file].values()]

# creat a Dataframe called  dataset_audiofilename_transcript_accent contains all audiofilename and thier 'transcript' and 'test_file' /Accent
    list_test_accent_txt_values.extend(list_test_accent_txt_values_temp)
    list_test_accent_txt_keys.extend(list_test_accent_txt_keys_temp)
    list_test_file.extend(list_test_file_temp)
    keys=['audiofilename','transcript','test_file']
    trans_dict_test_file_result = dict(zip(keys,[list_test_accent_txt_keys, list_test_accent_txt_values,list_test_file]))
    
dataset_audiofilename_transcript_accent = pd.DataFrame(trans_dict_test_file_result)

#############################################################################################
##### Show and write which Audio files and which Accent the given word is located ###########

for key, value in dataset_audiofilename_transcript_accent['transcript'].items():
  if required_word in value:

# to show the full name accent of the founded result 
    for keys_accent_long_i, values_accent_long_i in data_dict_accent_duration.items():
      if dataset_audiofilename_transcript_accent.test_file[key]==keys_accent_long_i:
        print()
        # print(dataset_audiofilename_transcript_accent.audiofilename[key],dataset_audiofilename_transcript_accent.transcript[key],
        # f'*** Accent ={dataset_audiofilename_transcript_accent.test_file[key]}')
        print(dataset_audiofilename_transcript_accent.audiofilename[key],dataset_audiofilename_transcript_accent.transcript[key],
              f'*** Accent is {data_dict_accent_duration[keys_accent_long_i]}')
 

        # # Alternative way (for speed performance) to export the Audio files numbers and thiers Acccent to an external file
        # # os.mkdir('/content/audiofilenames_transcript')

        # if newfile==0:
        #   dataset_accent_write_file = open(f'audiofilenames_transcript_accent_of_an_utterance.txt','w')
        #   newfile=1
        # else:
        #   dataset_accent_write_file = open(f'audiofilenames_transcript_accent_of_an_utterance.txt','a')
        #   dataset_accent_write_file.write(dataset_audiofilename_transcript_accent.audiofilename[key],dataset_audiofilename_transcript_accent.transcript[key],
        #       f'*** Accent is {data_dict_accent_duration[keys_accent_long_i]}\n')
        #   dataset_accent_write_file.close()
        #   # ! cp /content/audiofilenames_transcript/*.txt /content/drive/MyDrive/QU-DFKI-Thesis-ASR/Experimentation/data/test_data/audiofilenames_transcript/


common_voice_de_20537788 sie gründete die firma cochran’s crescent washing machine company die später in hobart aufging *** Accent is Österreichisches Deutsch

common_voice_de_23353340 die leihe wurde später um eine saison verlängert *** Accent is Österreichisches Deutsch

common_voice_de_25782562 die zweige sind erst grün und später rötlich braun *** Accent is Österreichisches Deutsch

common_voice_de_25778143 lee ordinierte ihn später und er war sein vorbild *** Accent is Österreichisches Deutsch

common_voice_de_26042371 später in dieser saison gewannen sie den lancashire combination cup *** Accent is Österreichisches Deutsch

common_voice_de_23353386 während des dreißigjährigen krieges wurde der ort fast gänzlich zerstört später wieder neu aufgebaut *** Accent is Österreichisches Deutsch

common_voice_de_24126891 mit der freien pistole gewann er drei tage später die silbermedaille *** Accent is Österreichisches Deutsch

common_voice_de_19504197 später kam die anlage in privatbesit